In [1]:
# Dependencies and Setup
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd

In [2]:
# Set Executable Path & Initialize Chrome Browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Visit the NASA Mars News Site
url = "https://mars.nasa.gov/news/"
browser.visit(url)

# Parse Results HTML with BeautifulSoup
html = browser.html
news_soup = BeautifulSoup(html, "html.parser")
slide_element = news_soup.select_one("ul.item_list li.slide")
slide_element.find("div", class_="content_title")
news_title = slide_element.find("div", class_="content_title").get_text()
news_paragraph = slide_element.find("div", class_="article_teaser_body").get_text()

# Display scrapped data 
print(news_title)
print(news_paragraph)

Watch NASA Build Its Next Mars Rover
A newly installed webcam offers the public a live, bird's-eye view of NASA's Mars 2020 rover as it takes shape at NASA's Jet Propulsion Laboratory. 


In [4]:
# Visit the url for JPL Featured Space Image
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)

# Scrape Path for the featured image starting with the partial path of the url
full_image_button = browser.find_by_id("full_image")
full_image_button.click()
browser.is_element_present_by_text("more info", wait_time=1)
more_info_element = browser.find_link_by_partial_text("more info")
more_info_element.click()

# Parse Results HTML with BeautifulSoup
html = browser.html
image_soup = BeautifulSoup(html, "html.parser")
img_url = image_soup.select_one("figure.lede a img").get("src")
img_url = f"https://www.jpl.nasa.gov{img_url}"

# Print to check the full URL
print(img_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA01322_hires.jpg


In [5]:
# Visit Mars Weather twitter for weather data
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

# Parse Results HTML with BeautifulSoup
html = browser.html
weather_soup = BeautifulSoup(html, "html.parser")
mars_weather_tweet = weather_soup.find("div", 
                                       attrs={
                                           "class": "tweet", 
                                            "data-name": "Mars Weather"
                                        })
mars_weather = mars_weather_tweet.find("p", "tweet-text").get_text()

# Print to check correct twitter data
print(mars_weather)

InSight sol 188 (2019-06-07) low -102.5ºC (-152.6ºF) high -21.9ºC (-7.4ºF)
winds from the SSE at 4.8 m/s (10.8 mph) gusting to 15.6 m/s (35.0 mph)
pressure at 7.60 hPapic.twitter.com/ocUTA1rgaU


In [6]:
# Visit the Mars Facts Site Using Pandas to Read
mars_df = pd.read_html("https://space-facts.com/mars/")[0]

mars_df.columns=["Description", "Value"]
mars_df.set_index("Description", inplace=True)

# Display the correct dataframe
mars_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [7]:
# Visit the USGS Astrogeology Science Center Site
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

hemisphere_image_urls = []

# Get a List of All the Hemispheres
links = browser.find_by_css("a.product-item h3")
for item in range(len(links)):
    hemisphere = {}
    
    # Find Element on Each Loop to Avoid a Stale Element Exception
    browser.find_by_css("a.product-item h3")[item].click()
    
    # Find Sample Image Anchor Tag & Extract <href>
    sample_element = browser.find_link_by_text("Sample").first
    hemisphere["img_url"] = sample_element["href"]
    
    # Get Hemisphere Title
    hemisphere["title"] = browser.find_by_css("h2.title").text
    
    # Append Hemisphere Object to List
    hemisphere_image_urls.append(hemisphere)
    
    # Navigate Backwards
    browser.back()

# Check for the correct image URLs
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [8]:
browser.quit()